In [1]:
# Install miscellaneous libraries.
!pip install torchdata
!pip install torch
!pip install transformers
!pip install torchvision
!pip install torchtext
!pip install torchaudio

# !pip install torchdata==0.4.1
# !pip install torch==1.12.1 
# !pip install transformers
# !pip install torchvision==0.13.1
# !pip install torchtext==0.13.1
# !pip install torchaudio==0.12.1

^C
ERROR: Operation cancelled by user


In [2]:
# Import libraries used throughout - if you need other libraries, 
# you are free to import them.
import functools
import random
import math
from queue import Queue
from typing import Any

import numpy as np
import torch
import torchtext
import torchtext.functional as F
import torchtext.transforms
from torch import nn
from torch.utils.data import DataLoader
from torchtext.datasets import UDPOS
from torchtext.vocab import build_vocab_from_iterator
from transformers import BertModel, BertTokenizer

In [3]:
# Constants and hyperparameters - you are free to change these for
# the bonus question.
SEED = 42
BATCH_SIZE = 32
EPOCHS = 3
LR = 2e-5
TRANSFORMER = "bert-base-uncased"

In [5]:
# Reproducibility.
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.use_deterministic_algorithms(True)

In [6]:
# Setting up dataloaders for training.
tokenizer = BertTokenizer.from_pretrained(TRANSFORMER)
init_token = tokenizer.cls_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
sep_token = tokenizer.sep_token
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)
sep_token_idx = tokenizer.convert_tokens_to_ids(sep_token)
max_input_length = tokenizer.max_model_input_sizes[TRANSFORMER]

train_datapipe = UDPOS(split="train")
valid_datapipe = UDPOS(split="valid")
pos_vocab = build_vocab_from_iterator(
    [i[1] for i in list(train_datapipe)],
    specials=[init_token, pad_token, sep_token],
)


def prepare_words(tokens, tokenizer, max_input_length, init_token, sep_token):
    """Preprocesses words such that they may be passed into BERT.

    Parameters
    ---
    tokens : List
        List of strings, each of which corresponds to one token in a sequence.
    tokenizer : transformers.models.bert.tokenization_bert.BertTokenizer
        Tokenizer to be used for transforming word strings into word indices
        to be used with BERT.
    max_input_length : int
        Maximum input length of each sequence as expected by our version of BERT.
    init_token : str
        String representation of the beginning of sentence marker for our tokenizer.
    sep_token : str
        String representation of the end of sentence marker for our tokenizer.

    Returns
    ---
    tokens : List
        List of preprocessed tokens.
    """
    # Append beginning of sentence and end of sentence markers
    # lowercase each token and cut them to the maximum length
    # (minus two to account for beginning and end of sentence).
    tokens = (
        [init_token]
        + [i.lower() for i in tokens[: max_input_length - 2]]
        + [sep_token]
    )
    # Convert word strings to indices.
    tokens = tokenizer.convert_tokens_to_ids(tokens)
    return tokens


def prepare_tags(tokens, max_input_length, init_token, sep_token):
    """Convert tag strings into indices for use with torch. For symmetry, we perform
        identical preprocessing as on our words, even though we do not need beginning
        of sentence and end of sentence markers for our tags.

    Parameters
    ---
    tokens : List
        List of strings, each of which corresponds to one token in a sequence.
    max_input_length : int
        Maximum input length of each sequence as expected by our version of BERT.
    init_token : str
        String representation of the beginning of sentence marker for our tokenizer.
    sep_token : str
        String representation of the end of sentence marker for our tokenizer.

    Returns
    ---
    tokens : List
        List of preprocessed tags.
    """
    # Append beginning of sentence and end of sentence markers
    # cut the tagging sequence to the maximum length (minus two to account for beginning and end of sentence).
    tokens = [init_token] + tokens[: max_input_length - 2] + [sep_token]
    # Convert tag strings to indices.
    tokens = torchtext.transforms.VocabTransform(pos_vocab)(tokens)
    return tokens


text_preprocessor = functools.partial(
    prepare_words,
    tokenizer=tokenizer,
    max_input_length=max_input_length,
    init_token=init_token,
    sep_token=sep_token,
)

tag_preprocessor = functools.partial(
    prepare_tags,
    max_input_length=max_input_length,
    init_token=init_token,
    sep_token=sep_token,
)


def apply_transform(x):
    return text_preprocessor(x[0]), tag_preprocessor(x[1])

In [7]:
train_datapipe = (
    train_datapipe.map(apply_transform)
    .batch(BATCH_SIZE)
    .rows2columnar(["words", "pos"])
)
train_dataloader = DataLoader(train_datapipe, batch_size=None, shuffle=False)
valid_datapipe = (
    valid_datapipe.map(apply_transform)
    .batch(BATCH_SIZE)
    .rows2columnar(["words", "pos"])
)
valid_dataloader = DataLoader(valid_datapipe, batch_size=None, shuffle=False)

In [8]:
class PriorityQueue(Queue):
    '''Variant of Queue that retrieves open entries in priority order (lowest first).

    Entries are typically tuples of the form:  (priority number, data).
    '''

    def _init(self, maxsize):
        self.queue = []
        self.keys = []

    def __str__ (self):
        res = ''
        for i in self.queue:
            res += " " + str(i)
        return res

    def _qsize(self):
        return len(self.queue)

    def _put(self, item):
        if item[0] in self.keys:
            index = -1
            for i in range(len(self.queue)):
                if self.queue[i][0] == item[0]:
                    index = i
            if index != -1:
                self.queue[index] = min(item, self.queue[index]) 
        else:
            self.keys.append(item[0])
            self.heappush(self.queue, item)

    def heappush(self, heap, item):
        """Push item onto heap, maintaining the heap invariant."""
        heap.append(item)
        self._siftdown(heap, 0, len(heap)-1)

    def _siftdown(self, heap, startpos, pos):
        newitem = heap[pos]
        # Follow the path to the root, moving parents down until finding a place
        # newitem fits.
        while pos > startpos:
            parentpos = (pos - 1) >> 1
            parent = heap[parentpos]
            if newitem[1] < parent[1]:
                heap[pos] = parent
                pos = parentpos
                continue
            break
        heap[pos] = newitem

    def _get(self):
        return self.heappop(self.queue)

    def heappop(self, heap):
        """Pop the smallest item off the heap, maintaining the heap invariant."""
        lastelt = heap.pop()    # raises appropriate IndexError if heap is empty
        if heap:
            returnitem = heap[0]
            heap[0] = lastelt
            self._siftup(heap, 0)
            return returnitem
        return lastelt
    
    def _siftup(self, heap, pos):
        endpos = len(heap)
        startpos = pos
        newitem = heap[pos]
        # Bubble up the smaller child until hitting a leaf.
        childpos = 2*pos + 1    # leftmost child position
        while childpos < endpos:
            # Set childpos to index of smaller child.
            rightpos = childpos + 1
            if rightpos < endpos and not heap[childpos][1] < heap[rightpos][1]:
                childpos = rightpos
            # Move the smaller child up.
            heap[pos] = heap[childpos]
            pos = childpos
            childpos = 2*pos + 1
        # The leaf at pos is empty now.  Put newitem there, and bubble it up
        # to its final resting place (by sifting its parents down).
        heap[pos] = newitem
        self._siftdown(heap, startpos, pos)

In [9]:
class TagLSTM(nn.Module):
    """Models an LSTM on top of a transformer to predict POS in a Neural CRF."""

    def __init__(self, nb_labels, emb_dim, hidden_dim=256):
        """Constructor.

        Parameters
        ---
        nb_labels : int
            Number of POS tags to be considered.

        emb_dim : int
            Input_size of the LSTM - effectively embedding dimension of our pretrained transformer.

        hidden_dim : int
            Hidden dimension of the LSTM.
        """
        super().__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            emb_dim, hidden_dim // 2, bidirectional=True, batch_first=True
        )
        self.tag = nn.Linear(hidden_dim, nb_labels)
        self.hidden = None

    def init_hidden(self, batch_size):
        return (
            torch.randn(2, batch_size, self.hidden_dim // 2),
            torch.randn(2, batch_size, self.hidden_dim // 2),
        )

    def forward(self, x):
        self.hidden = self.init_hidden(x.shape[0])
        x, self.hidden = self.lstm(x, self.hidden)
        x = self.tag(x)
        return x


class NeuralCRF(nn.Module):
    """Class modeling a neural CRF for POS tagging.
    We model tag-tag dependencies with a weight for each transition
    and word-tag influence through an LSTM on top of a pretrained transformer.
    """

    def __init__(
        self,
        pad_idx_word,
        pad_idx_pos,
        bos_idx,
        eos_idx,
        bot_idx,
        eot_idx,
        t_cal,
        transformer,
        lstm_hidden_dim=64,
        beta=0,
    ):
        """Constructor.

        Parameters
        ---
        pad_idx_word : int
            Index corresponding to padding in the word sequences.
        pad_idx_pos : int
            Index corresponding to padding in the tag sequences.
        bos_idx : int
            Index corresponding to beginning of speech marker in the word sequences.
        eos_idx : int
            Index corresponding to end of speech marker in the word sequences.
        bot_idx : int
            Index corresponding to beginning of tag marker in the tag sequences.
        eot_idx : int
            Index corresponding to end of tag marker in the tag sequences.
        t_cal : List[int]
            List containing all indices corresponding to tags in the tag sequences.
        transformer : BertModel
            Pretrained transformer used to embed sentences before feeding them
            into the LSTM.
        lstm_hiden_dim : int
            Hidden dimension of the LSTM used for POS tagging. Note that
            since we are bidirectional, the effective hidden dimension
            is half of this number.
        beta : float
            Regularization hyperparameter of the entropy regularizer.
            Entropy regularization is only applied for \beta > 0.
        """
        super().__init__()
        self.pad_idx_word = pad_idx_word
        self.pad_idx_pos = pad_idx_pos
        self.bos_idx = bos_idx
        self.eos_idx = eos_idx
        self.bot_idx = bot_idx
        self.eot_idx = eot_idx
        self.t_cal = t_cal
        self.transformer = transformer
        self.lstm_hidden_dim = lstm_hidden_dim
        self.beta = beta
        self.transitions = nn.Parameter(torch.empty(len(t_cal), len(t_cal)))
        self.emissions = TagLSTM(
            len(t_cal),
            transformer.config.to_dict()["hidden_size"],
            lstm_hidden_dim,
        )
        self.init_weights()

    def init_weights(self):
        nn.init.uniform_(self.transitions, -0.1, 0.1)

    def forward(self, W):
        """Decode each sentence within W and return predicted tagging.

        Parameters
        ---
        W : torch.tensor
            Word sequences of dimension batch size x max sentence length within batch + 2.

        Returns
        ---
        sequences : list
            List of tensors, each of which contains the predicted tag indices for a particular
            word sequence.
        """
        # Calculate scores.
        emissions = self.calculate_emissions(W)
        # Run viterbi sentence by sentence.
        sequences = []
        for sentence in range(W.shape[0]):
            # Exclude beginning and end markers from each word sequence.
            scores, backpointers = self.backward_viterbi_log(
                W[sentence, 1:], emissions[sentence, :]
            )
            sequences += [self.get_viterbi(backpointers)]
        return sequences

    def calculate_emissions(self, W):
        """Calculate emissions (i.e., scores for each word and batch).

        Parameters
        ---
        W : torch.tensor
            Word sequences of dimension batch size x max sentence
            length within batch + 2.

        Returns
        ---
        emissions : torch.tensor
            Word level scores for each tag of dimension batch_size x max
            sentence length within batch + 1 x |T|.
            The scores for the initial BOS index are already removed here
            since we only needed it for the transformer.
        """
        # Directly exclude emissions for the initial word in each sentence
        # since these correspond to the BOS indices that we only need
        # for BERT.
        return self.emissions(self.transformer(W)[0])[:, 1:, :]

    def loss(self, T, W):
        """Calculate the loss for a batch.

        Parameters
        ---
        T : torch.tensor
            True taggings for each sequence within the batch.
            Of dimension batch size x longest sequence within batch + 2.
            Note the paddings, EOS and BOS that have been added to T
            for symmetry with W which needs this for BERT.
        W : torch.tensor
            Words for each sequence within the batch.
            Of dimension batch size x longest sequence within batch + 2.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT.

        Returns
        ---
        torch.tensor
            Mean loss for the batch.
        """
        emissions = self.calculate_emissions(W)
        # Note that we have to handle paddings and EOS within the score
        # and backward functions, but we can already skip the BOS tokens
        # here.
        scores = self.score(emissions, W[:, 1:], T[:, 1:])
        log_normalizer = self.backward_log_Z(W[:, 1:], emissions)
        # NB: If you don't calculate your normaliser in log-space,
        # you may have issues with normalisers being too large. 
        # Since this occurs very rarely, you can just skip those 
        # batches with a high pseudo normaliser (comment out the code below).

        #if torch.isinf(log_normalizer) or torch.isnan(log_normalizer):
        #   log_normalizer = torch.log(torch.tensor(1e+200, dtype=torch.float64))
        loss = torch.negative(torch.mean(scores - log_normalizer))
        if self.beta > 0.0:
            unnormalized_entropy = self.backward_entropy(
                W[:, 1:], emissions
            )
            entropy = (
                (unnormalized_entropy / torch.exp(log_normalizer))
                + log_normalizer
            )
            if torch.isinf(torch.max(torch.exp(log_normalizer))):
                return loss
            else:
                return loss + torch.negative(self.beta * torch.mean(entropy))
        else:
            return loss

    def score(self, emissions, W, T):
        """Calculate scores for specified taggings and word sequences.

        Parameters
        ---
        emissions : torch.tensor
        T : torch.tensor
            Taggings for each sequence within the batch.
            Of dimension batch size x longest sequence within batch + 1.
            Note the paddings, EOS and BOS that have been added to T
            for symmetry with W which needs this for BERT.
            We expect T to already have the initial BOT tag indices removed
            (see `loss` for details).
        W : torch.tensor
            Words for each sequence within the batch.
            Of dimension batch size x longest sequence within batch + 1.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT so we mask them out here. We expect
            W to already have the initial BOS word indices taken out
            (see `loss` for details).

        Returns
        ---
        scores : torch.tensor
            score(T, W) for all samples in W.
        """
        scores = (
            emissions[:, 0].gather(1, (T[:, 0]).unsqueeze(1)).squeeze()
            + self.transitions[self.bot_idx, T[:, 0]]
        )
        for word in range(1, emissions.shape[1]):
            mask = torch.where(
                W[:, word] == self.pad_idx_word, 0, 1
            ) * torch.where(W[:, word] == self.eos_idx, 0, 1)
            scores += mask * (
                emissions[:, word]
                .gather(1, (T[:, word]).unsqueeze(1))
                .squeeze()
                + self.transitions[T[:, word - 1], T[:, word]]
            )
        return scores

    def viterbi_naive(self, W, emissions):
        """Calculate best tagging naively and return both the best score and best tagging in log space.

        NB: This naive version is not vectorized over samples.

        Parameters
        ---
        W : torch.tensor
            Of dimension longest sequence within batch + 2 or less.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT so we manually remove them here if present.
        emissions : torch.tensor
            Word level scores for each tag of dimension max
            sentence length within batch + 1 x |T| (we assume scores for the BOT
            initial tag have already been removed since BOT/BOS is
            only needed for the transformer).

        Returns
        ---
        Tuple[torch.tensor, torch.tensor]
            Tuple containing the log-score of the best tagging and the
            indices of the best tagging for W.
        """
        T = self.t_cal
        # Remove padding.
        if torch.any(W == self.pad_idx_word):
            W = W[torch.where(W != self.pad_idx_word)[0]]
        # Remove EOS and BOS if present.
        if torch.any(W == self.eos_idx):
            W = W[:-1]
        if torch.any(W == self.bos_idx):
            W = W[1:]
        T_abs = len(T)
        combinations = torch.combinations(
            T, r=W.shape[0], with_replacement=True
        )
        combinations = torch.cartesian_prod(*[T for ix in range(W.shape[0])])
        best_score = torch.tensor(0.0, dtype=torch.float64)
        best_tag = torch.tensor([])
        for ix, combination in enumerate(combinations):
            if W.shape[0] == 1:
                current_score = (
                    emissions[0, combination]
                    + self.transitions[self.bot_idx, combination]
                )
            else:
                current_score = (
                    emissions[0, combination[0]]
                    + self.transitions[self.bot_idx, combination[0]]
                )
                for qx in range(1, combination.shape[0]):
                    current_score += (
                        emissions[qx, combination[qx]]
                        + self.transitions[
                            combination[qx - 1], combination[qx]
                        ]
                    )

            if (current_score) > best_score:
                best_score = current_score.double()
                best_tag = combination
        return best_score, best_tag

    def log_Z_naive(self, W, emissions):
        """Calculate log Z naively.

        NB: This naive version is not vectorized over samples.

        Parameters
        ---
        W : torch.tensor
            Of dimension longest sequence within batch + 2 or less.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT so we manually remove them here if present.
        emissions : torch.tensor
            Word level scores for each tag of dimension max
            sentence length within batch + 1 x |T| (we assume scores for the BOT; So the + 1 refers to the EOT tag?
            initial tag have already been removed since BOT/BOS is
            only needed for the transformer).

        Returns
        ---
        torch.tensor
            Log Z for W.
        """
        T = self.t_cal
        # Remove padding
        W = W[torch.where(W != self.pad_idx_word)[0]]
        # Remove EOS and BOS if present
        if torch.any(W == self.eos_idx):
            W = W[:-1]
        if torch.any(W == self.bos_idx):
            W = W[1:]
        T_abs = len(T)
        
        # Generate \mathcal{T}^N.
        #This represents the possible sequences of tags,
        #So the shape should be [|T|^(W.shape[0]), W.shape[0]], no?
        combinations = torch.cartesian_prod(*[T for ix in range(W.shape[0])])
        #With shape [|T|^(W.shape[0])], no?
        log_normalizer = torch.zeros(
            combinations.shape[0], dtype=torch.float64
        )
        # Loop over all possible combinations naively.
        # NB: This is essentially line one on Slide 50.
        for ix, combination in enumerate(combinations):
            # Kludge since indexing is slightly different for one-dim
            # tensors vs two tensors.
            if W.shape[0] == 1:
                # Calculate score as the sum of emissions (i.e., how well
                # does a word match a tag based on BERT embeddings) and
                # transitions (globally, how likely is a transition
                # from the previous tag to the current tag).
                # NB: For the first word, the initial tag is always BOT.
                # NB 2: Since we are in log-space, the exp
                # of the score goes away.
                log_normalizer[ix] = (
                    emissions[0, combination]
                    + self.transitions[self.bot_idx, combination]
                )

            else:
                # Initial score is identical to above.
                log_normalizer[ix] = (
                    emissions[0, combination[0]]
                    + self.transitions[self.bot_idx, combination[0]]
                )
                for qx in range(1, combination.shape[0]):
                    # Score within each potential tagging
                    # is calculated the same as above except that we now 
                    # actually use the previous tag instead of always
                    # BOT.
                    log_normalizer[ix] += (
                        emissions[qx, combination[qx]]
                        + self.transitions[
                            combination[qx - 1], combination[qx]
                        ]
                    )
        # Calculate logsumexp numerically stable
        # since we are in log-space.
        return torch.logsumexp(log_normalizer, 0)


    def entropy_naive(self, W, emissions):
        """Calculate the unnormalized entropy naively.

        NB: This naive version is not vectorized over samples.

        Parameters
        ---
        W : torch.tensor
            Words for each sequence within the batch.
            Of dimension longest sequence within batch + 2 or less.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT so we manually remove them here if present.
        emissions : torch.tensor
            Word level scores for each tag of dimension max
            sentence length within batch + 1 x |T| (we assume scores for the BOT
            initial tag have already been removed since BOT/BOS is
            only needed for the transformer).

        Returns
        ---
        torch.tensor
            Log Z for W.
        """
        T = self.t_cal
        # Remove padding
        W = W[torch.where(W != self.pad_idx_word)[0]]
        # Remove EOS and BOS if present
        if torch.any(W == self.eos_idx):
            W = W[:-1]
        if torch.any(W == self.bos_idx):
            W = W[1:]
        T_abs = len(T)
        combinations = torch.combinations(
            T, r=W.shape[0], with_replacement=True
        )
        combinations = torch.cartesian_prod(T, T)
        combinations = torch.cartesian_prod(*[T for ix in range(W.shape[0])])
        entropy = torch.zeros(1, dtype=torch.float64)
        for ix, combination in enumerate(combinations):
            if W.shape[0] == 1:
                entropy -= torch.exp((
                    emissions[0, combination]
                    + self.transitions[self.bot_idx, combination]
                )) * (
                    emissions[0, combination]
                    + self.transitions[self.bot_idx, combination]
                )

            else:
                local_score = (
                    emissions[0, combination[0]]
                    + self.transitions[self.bot_idx, combination[0]]
                )
                for qx in range(1, combination.shape[0]):
                    local_score += (
                        emissions[qx, combination[qx]]
                        + self.transitions[
                            combination[qx - 1], combination[qx]
                        ]
                    )
                entropy -= torch.exp(local_score) * local_score
        return entropy

    def get_viterbi(self, backpointer_matrix):
        """Return the best tagging based on a backpointer matrix.

        Parameters
        ---
        backpointer_matrix : torch.tensor
            Backpointer matrix from Viterbi indicating which
            tag is the highest scoring for each element in the sequence.

        Returns
        ---
        torch.tensor
            Indices of the best tagging based on `backpointer_matrix`.
        """
        T = self.t_cal
        
        initial_tag = backpointer_matrix[0,0].item()
        best_tagging = torch.tensor([T[int(initial_tag)]])
        
        for i in range(1, backpointer_matrix.shape[0]):
          initial_tag = backpointer_matrix[i,int(initial_tag)].item()
          best_tagging = torch.cat((best_tagging, T[int(initial_tag)].reshape((1))))
        
        return best_tagging

    def backward_log_Z(self, W, emissions):
        """Calculate log Z using the backward algorithm.

        NB: You do need to vectorize this over samples.

        Parameters
        ---
        W : torch.tensor
            Words for each sequence within the batch.
            Of dimension batch size x longest sequence within batch + 1.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT so we mask them out here. We expect
            W to already have the initial BOS word indices taken out.
        emissions : torch.tensor
            Word level scores for each tag of dimension batch_size x max
            sentence length within batch + 1 x |T| (scores for the BOS
            initial tag have already been removed since BOS is
            only needed for the transformer).

        Returns
        ---
        torch.tensor
            Log Z for each sample in W.
        """
        T = self.t_cal
        T_abs = len(T)  
        backward_var = torch.zeros((W.shape[0], W.shape[1], T_abs), dtype=torch.float64)
        mask = torch.where((W == self.eos_idx) | (W == self.bos_idx) | (W == self.pad_idx_word), 0., 1.) 
        
        for word_index in range(W.shape[1] - 2, -1, -1):
            for tag_index in range(T_abs):
              score = self.transitions[T[tag_index], :][None, :] + emissions[:, word_index + 1, :] + backward_var[:, word_index + 1, :]#Try word_index + 1 Automatic broadcast ovet the size since the number of rows should be the same
              backward_var[:, word_index, tag_index] = torch.logsumexp(score, 1) * mask[:, word_index + 1] + (1 - mask[:, word_index + 1]) * backward_var[:, word_index + 1, tag_index] #Specifica The sum over the row
        log_Z = self.transitions[self.bot_idx, :][None, :] + emissions[:, 0, :] + backward_var[:, 0, :]
        return torch.logsumexp(log_Z, 1)

        # result = torch.zeros(W.shape[0], dtype=torch.float64)
        # for batch_num in range(W.shape[0]):
        #   sentence = W[batch_num]
        #   T = self.t_cal
        #   # Remove padding
        #   sentence = sentence[torch.where(sentence != self.pad_idx_word)[0]]
        #   # Remove EOS and BOS if present
        #   if torch.any(sentence == self.eos_idx):
        #       sentence = sentence[:-1]
        #   if torch.any(sentence == self.bos_idx):
        #       sentence = sentence[1:]
        #   T_abs = len(T)

        #   backward_var = torch.zeros((sentence.shape[0], T_abs), dtype=torch.float64)
  
        #   backward_var[sentence.shape[0] - 1, :] = 1        
        #   for word_index in range(sentence.shape[0] - 2, -1, -1):
        #     score = self.transitions[T[:]] + emissions[batch_num, word_index + 1, T[:]] #Try word_index + 1 Automatic broadcast ovet the size since the number of rows should be the same
        #     backward_var[word_index, :] += torch.sum(torch.exp(score)*backward_var[word_index + 1, :], dim=1) #The sum over the row
        #   log_Z = 0
        #   score = self.transitions[self.bot_idx, T[:]] + emissions[batch_num, 0, T[:]]
        #   log_Z += torch.sum(torch.exp(score)*backward_var[0][:], dim=0)

        #   log_Z = math.log(log_Z)
        #   result[batch_num] = log_Z

        # return result


    def forward_log_Z(self, W, emissions):
        """Calculate log Z using the forward algorithm.

        NB: You do need to vectorize this over samples.

        Parameters
        ---
        W : torch.tensor
            Words for each sequence within the batch.
            Of dimension batch size x longest sequence within batch + 1.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT so we mask them out here. We expect
            W to already have the initial BOS word indices taken out.
        emissions : torch.tensor
            Word level scores for each tag of dimension batch_size x max
            sentence length within batch + 1 x |T| (scores for the BOS
            initial tag have already been removed since BOS is
            only needed for the transformer).

        Returns
        ---
        torch.tensor
            Log Z for each sample in W.
        """
        result = torch.zeros(W.shape[0], dtype=torch.float64)
        for batch_num in range(W.shape[0]):
          sentence = W[batch_num]
          T = self.t_cal
          # Remove padding
          sentence = sentence[torch.where(sentence != self.pad_idx_word)[0]]
          # Remove EOS and BOS if present
          if torch.any(sentence == self.eos_idx):
              sentence = sentence[:-1]
          if torch.any(W == self.bos_idx):
              sentence = sentence[1:]
          T_abs = len(T)

          forward_var = torch.zeros((sentence.shape[0], T_abs), dtype=torch.float64)
          
          for i in range(T_abs):
            score = self.transitions[self.bot_idx, T[i]] + emissions[batch_num, 0, T[i]]
            forward_var[0][i] = math.exp(score)
          
          for word_index in range(1, sentence.shape[0]):
            for tag_index in range(T_abs):
              for previews_tag_index in range(T_abs):
                score = self.transitions[T[previews_tag_index], T[tag_index]] + emissions[batch_num, word_index, T[tag_index]] #this is word_index - 1
                forward_var[word_index, tag_index] += math.exp(score)*forward_var[word_index - 1, previews_tag_index] 
          
          log_Z = 0
          for i in range(T_abs):
            log_Z += forward_var[sentence.shape[0] - 1, i] #Probabilmente il problema è qui :/ devo chiedere
          
          log_Z = math.log(log_Z)
          result[batch_num] = log_Z

        return result

    def backward_entropy(self, W, emissions):
        """Calculate the unnormalized entropy using the backward algorithm.

        NB: You do need to vectorize this over samples.

        Parameters
        ---
        W : torch.tensor
            Words for each sequence within the batch.
            Of dimension batch size x longest sequence within batch + 1.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT so we mask them out here. We expect
            W to already have the initial BOS word indices taken out.
        emissions : torch.tensor
            Word level scores for each tag of dimension batch_size x max
            sentence length within batch + 1 x |T| (scores for the EOS
            initial tag have already been removed since EOS is
            only needed for the transformer).

        Returns
        ---
        torch.tensor
            Unnormalized entropy for each sample in W.
        """

        T = self.t_cal
        T_abs = len(T)  
        backward_var = torch.zeros((W.shape[0], W.shape[1], T_abs, 2), dtype=torch.float64)
        mask = torch.where((W == self.eos_idx) | (W == self.bos_idx) | (W == self.pad_idx_word), 0., 1.) 
        
        backward_var[:, W.shape[1] - 1, :, 0] = 1 
        backward_var[:, W.shape[1] - 1, :, 1] = 0
        
        transitions_temp = self.transitions.clone().detach()
        emissions_temp = emissions.clone().detach()

        for word_index in range(W.shape[1] - 2, -1, -1):
            for tag_index in range(T_abs):
              score_temp1 = transitions_temp[T[tag_index], T[:]] + emissions_temp[:, word_index + 1, :] #Try word_index + 1
              score_temp = torch.exp(score_temp1)
              
              score_vec = [score_temp, -score_temp*score_temp1]
              score = [score_vec[0]*backward_var.clone().detach()[:, word_index + 1, :, 0], score_vec[0]*backward_var.clone().detach()[:, word_index + 1, :, 1] + score_vec[1]*backward_var.clone().detach()[:, word_index + 1, :, 0]]
              
              backward_var[:, word_index, tag_index, 0] = torch.sum(score[0], dim=1) * mask[:, word_index + 1] + backward_var.clone().detach()[:, word_index + 1, tag_index, 0] * (1 - mask[:, word_index + 1]) 
              backward_var[:, word_index, tag_index, 1] = torch.sum(score[1], dim=1) * mask[:, word_index + 1] + backward_var.clone().detach()[:, word_index + 1, tag_index, 1] * (1 - mask[:, word_index + 1])     
          
        log_Z = [0, 0]

        score_temp1 = transitions_temp[self.bot_idx, T[:]] + emissions_temp[:, 0, :]
        score_temp = torch.exp(score_temp1)
        score_vec = [score_temp, -score_temp*score_temp1] 

        score = [score_vec[0]*backward_var.clone().detach()[:, 0, :, 0], score_vec[0]*backward_var.clone().detach()[:, 0, :, 1] + score_vec[1]*backward_var.clone().detach()[:, 0, :, 0]] 

        #log_Z[0] += torch.sum(score[0], dim=1)
        log_Z[1] += torch.sum(score[1], dim=-1)
        
        return log_Z[1]

        # result = torch.zeros(W.shape[0], dtype=torch.float64)
        # for batch_num in range(W.shape[0]):
        #   sentence = W[batch_num]
        #   T = self.t_cal
        #   # Remove padding
        #   sentence = sentence[torch.where(sentence != self.pad_idx_word)[0]]
        #   # Remove EOS and BOS if present
        #   if torch.any(sentence == self.eos_idx):
        #       sentence = sentence[:-1]
        #   if torch.any(W == self.bos_idx):
        #       sentence = sentence[1:]
        #   T_abs = len(T)

        #   backward_var = torch.zeros((sentence.shape[0], T_abs, 2), dtype=torch.float64)
          
        #   backward_var[sentence.shape[0] - 1, :, 0] = 1 
        #   backward_var[sentence.shape[0] - 1, :, 1] = 0
          
        #   for word_index in range(sentence.shape[0] - 2, -1, -1):
            
        #     score_temp1 = self.transitions[T[:]] + emissions[batch_num, word_index + 1, T[:]] #Try word_index + 1
        #     score_temp = torch.exp(score_temp1)
            
        #     score_vec = [score_temp, -score_temp*score_temp1]
        #     score = [score_vec[0]*backward_var[word_index + 1, :, 0], score_vec[0]*backward_var[word_index + 1, :,1] + score_vec[1]*backward_var[word_index + 1, :, 0]]
            
        #     backward_var[word_index, :, 0] += torch.sum(score[0], dim=1)
        #     backward_var[word_index, :, 1] += torch.sum(score[1], dim=1)
          
        #   log_Z = [0, 0]

        #   score_temp1 = self.transitions[self.bot_idx, T[:]] + emissions[batch_num, 0, T[:]]
        #   score_temp = torch.exp(score_temp1)
        #   score_vec = [score_temp, -score_temp*score_temp1] 

        #   score = [score_vec[0]*backward_var[0, :, 0], score_vec[0]*backward_var[0, :, 1] + score_vec[1]*backward_var[0, :, 0]] 

        #   log_Z[0] += torch.sum(score[0], dim=0)
        #   log_Z[1] += torch.sum(score[1], dim=0)
          
        #   result[batch_num] = log_Z[1]

        #   return result

    def backward_viterbi_log(self, W, emissions):
        """Calculate the best tagging using the backward algorithm and return
            both the scoring matrix in log-space and the backpointer matrix.

        NB: You do not need to vectorize this over samples.

        Parameters
        ---
        W : torch.tensor
            Of dimension longest sequence within batch + 2 or less.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT so we manually remove them here if present.
        emissions : torch.tensor
            Word level scores for each tag of dimension max
            sentence length within batch + 1 x |T| (we assume scores for the BOT
            initial tag have already been removed since BOT/BOS is
            only needed for the transformer).

        Returns
        ---
        Tuple[torch.tensor, torch.tensor]
            Tuple containing the scoring matrix in log-space and the
            backpointer matrix for recovering the best tagging.
        """
        
        sentence = W
        T = self.t_cal
        # Remove padding
        sentence = sentence[torch.where(sentence != self.pad_idx_word)[0]]
        # Remove EOS and BOS if present
        if torch.any(sentence == self.eos_idx):
            sentence = sentence[:-1]
        if torch.any(W == self.bos_idx):
            sentence = sentence[1:]
        T_abs = len(T)

        backward_var = torch.zeros((sentence.shape[0], T_abs), dtype=torch.float64)
        best_tag = torch.zeros((sentence.shape[0], T_abs))

        for i in range(T_abs):
          backward_var[sentence.shape[0] - 1][i] = 0
        
        for word_index in range(sentence.shape[0] - 2, -1, -1):
          for tag_index in range(T_abs):
            current_best_tag = None
            for next_tag_index in range(T_abs):
              score = self.transitions[T[tag_index], T[next_tag_index]] + emissions[word_index + 1, T[next_tag_index]] #Try word_index + 1
              
              if backward_var[word_index, tag_index] < score + backward_var[word_index + 1, next_tag_index]:
                backward_var[word_index, tag_index] = score + backward_var[word_index + 1, next_tag_index]
                current_best_tag = next_tag_index 
            
            best_tag[word_index + 1, tag_index] = current_best_tag

        
        final_best_score = 0
        current_best_tag = None
        
        for i in range(T_abs):
          score = self.transitions[self.bot_idx, T[i]] + emissions[0, T[i]]
          
          if final_best_score < score + backward_var[0][i]:
            final_best_score = score + backward_var[0][i]
            current_best_tag = i
        
        best_tag[0, 0] = current_best_tag
  
        return final_best_score, best_tag

    def dijkstra_viterbi_log(self, W, emissions):
        """Calculate the best tagging using Dijsktra's algorithm and return
            both the best score and best tagging in log space.

        NB: You do not need to vectorize this over samples.

        Parameters
        ---
        W : torch.tensor
            Of dimension longest sequence within batch + 2 or less.
            Note the paddings, EOS and BOS that have been added to W
            for usage with BERT so we manually remove them here if present.
        emissions : torch.tensor
            Word level scores for each tag of dimension max
            sentence length within batch + 1 x |T| (we assume scores for the BOT
            initial tag have already been removed since BOT/BOS is
            only needed for the transformer).


        Returns
        ---
        Tuple[torch.tensor, None, log_Z]
            Tuple containing the log-score of the best tagging. 
            NB: Since there were some changes in the assignment,
            we don't expect you to return the backpointer matrix
            this year.
            NB 2: We return log_Z if we already use it within the method
            to calculate probabilities, such that we don't have to
        """
        log_Z = self.backward_log_Z(torch.reshape(W, (1,W.shape[0])), 
                                      torch.reshape(emissions, (1,emissions.shape[0],emissions.shape[1])))
        sentence = W
        T = self.t_cal
        # Remove padding
        args_of_interest = torch.where(sentence != self.pad_idx_word)[0]
        sentence = sentence[args_of_interest]
        # Remove EOS and BOS if present
        if torch.any(sentence == self.eos_idx):
            sentence = sentence[:-1]
        if torch.any(W == self.bos_idx):
            sentence = sentence[1:]
        T_abs = len(T)
        
        result = torch.zeros((sentence.shape[0] + 1, T_abs), dtype=torch.float64) + float('-inf')
        queue = PriorityQueue()
        popped = []

        queue.put(((0,self.bot_idx), 0))

        while not queue.empty():
          key, score = queue.get()
          score = 0 - score # Perché viene inserito come -score cosi da ottenere lo score maggiore

          if key[0] == self.bot_idx:
            result[0, 0] = score
          else:
            popped.append(key)  
            result[key[0], key[1]] = score
          
          if key[0] < sentence.shape[0]:
            for t in range(T_abs):
              if (key[0] + 1, t) not in popped:
                if key[0] == 0:
                  weight = self.transitions[self.bot_idx, T[t]] + emissions[key[0], T[t]] #Try word_index + 1
                else:
                  weight = self.transitions[T[key[1]], T[t]] + emissions[key[0], T[t]] #Try word_index + 1
            
                queue.put(((key[0] + 1, t), -(weight + score) + log_Z))
        
        final_score = torch.tensor(float('-inf'))
        for t in range(T_abs):
          final_score = torch.maximum(final_score, result[sentence.shape[0], t])        
        return final_score, None, log_Z


In [10]:
for i, data in enumerate(train_dataloader):
    W_train = F.to_tensor(data["words"], padding_value=pad_token_idx)
    T_train = F.to_tensor(data["pos"], padding_value=pos_vocab[pad_token])
    if i == 0:
        break

bert = BertModel.from_pretrained(TRANSFORMER)
T_CAL = torch.tensor([i for i in range(pos_vocab.__len__())])
crf = NeuralCRF(
    pad_idx_word=pad_token_idx,
    pad_idx_pos=pos_vocab[pad_token],
    bos_idx=init_token_idx,
    eos_idx=sep_token_idx,
    bot_idx=pos_vocab[init_token],
    eot_idx=pos_vocab[sep_token],
    t_cal=T_CAL,
    transformer=bert,
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Q3a)

In [ ]:
emissions = crf.calculate_emissions(W_train)

for sentence in range(W_train.shape[0]):
    for word_index in [2, 3, 4]:
        assert torch.isclose(
            crf.log_Z_naive(
                W_train[sentence, :word_index],
                emissions[
                    sentence,
                ],
            ),
            crf.backward_log_Z(W_train[:, 1:word_index], emissions)[sentence],
        )


In [ ]:
# NB: THIS TEST IS NOT GRADED! It is just to help you double check 
# your implementation of backward since the above test cases do not 
# contain instances of EOT/PAD.
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

crf = NeuralCRF(
    pad_idx_word=pad_token_idx,
    pad_idx_pos=pos_vocab[pad_token],
    bos_idx=init_token_idx,
    eos_idx=sep_token_idx,
    bot_idx=pos_vocab[init_token],
    eot_idx=pos_vocab[sep_token],
    t_cal=T_CAL,
    transformer=bert,
)
emissions = crf.calculate_emissions(W_train)
assert torch.all(torch.isclose(crf.backward_log_Z(W_train[:, 1:], emissions), 
        
        torch.tensor([ 88.3197,  54.9441,  51.8413,  49.4724, 110.4930,  40.2174,  40.0351,
         49.2687, 107.3545,  60.9668,  58.0769,  89.1132,  57.9029,  51.7252,
         83.1715,  79.5805,  82.7579,  33.7521,  61.2349,  27.4993,  45.7138,
         49.2352,  52.6344, 122.5854, 150.2306, 156.5732,  40.0482,  30.7149,
         27.3859,  92.0778,  61.3652,  80.0837], dtype=torch.float64)))

## Q3b)

In [ ]:
emissions = crf.calculate_emissions(W_train)
assert torch.all(
    torch.isclose(
        crf.backward_log_Z(W_train[:, 1:], emissions),
        crf.forward_log_Z(W_train[:, 1:], emissions),
    )
)


## Q3c)

In [ ]:
emissions = crf.calculate_emissions(W_train)

for sentence in range(W_train.shape[0]):
    for word_index in [2, 3, 4]:
        score_naive, sequence_naive = crf.viterbi_naive(
            W_train[sentence, :word_index],
            emissions[
                sentence,
            ],
        )
        score_viterbi, backpointers_viterbi = crf.backward_viterbi_log(
            W_train[sentence, :word_index],
            emissions[
                sentence,
            ],
        )
        sequence_viterbi = crf.get_viterbi(backpointers_viterbi)

        assert torch.isclose(score_viterbi, score_naive)
        assert torch.all(sequence_viterbi == sequence_naive)


In [ ]:
# NB: THIS TEST IS NOT GRADED! It is just to help you double check 
# your implementation of Viterbi since the above test cases do not 
# contain instances of EOT/PAD.
# NB2: Our evaluation expects Viterbi to only predict tags for actual 
# words, thus Viterbi (or get_viterbi) is supposed to remove instances
# of EOS/BOS/PAD. Example: If Viterbi is asked to predict the sequence 
# ["BOS", "I", "like" "dogs", "EOS", "PAD", "PAD"], it should return a tagging 
# of length 3 (one for each valid word in the input).
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

crf = NeuralCRF(
    pad_idx_word=pad_token_idx,
    pad_idx_pos=pos_vocab[pad_token],
    bos_idx=init_token_idx,
    eos_idx=sep_token_idx,
    bot_idx=pos_vocab[init_token],
    eot_idx=pos_vocab[sep_token],
    t_cal=T_CAL,
    transformer=bert,
)
emissions = crf.calculate_emissions(W_train)
sequences_overall = []
first_batch_sequences_test = [torch.tensor([10, 13, 10,  4, 13, 10, 10,  0,  0, 10, 10,  0, 10,  5,  4,  1,  5,  6,
         10,  4,  0, 10,  0, 10,  0, 10,  0, 19, 15]),
 torch.tensor([10, 10,  4,  8, 15, 14,  4, 14, 10,  4, 14, 10,  4, 14,  4,  4,  5,  5]),
 torch.tensor([10,  4, 13, 10,  4, 13, 10,  4, 13, 10,  4, 10,  4, 10,  4, 13, 10]),
 torch.tensor([10, 10,  5, 10,  4,  9,  4, 10,  4, 15, 10,  1,  6, 10,  6, 10]),
 torch.tensor([10, 10, 10, 13, 10,  4, 10,  6, 10, 13,  5, 10, 12, 10, 12, 14, 14,  6,
          6,  6, 10, 10, 10,  6, 10,  6, 10, 10,  6, 10,  7,  6, 10,  6, 18, 10]),
 torch.tensor([10,  4, 19, 15,  4, 15,  4, 15, 10,  0, 10,  4,  5]),
 torch.tensor([ 5,  5, 10, 10, 10,  6,  8, 15, 15, 10,  5, 10,  5]),
 torch.tensor([10,  4, 15, 14, 10,  4,  4, 15,  8, 10,  9,  4,  5,  5,  5,  5]),
 torch.tensor([10,  6, 10,  0, 10,  0, 10,  0, 10,  4,  0, 15, 10, 10,  0, 15,  5,  6,
         10,  4, 15, 14, 10, 10,  6, 10, 10, 10, 10,  6, 10,  0, 10,  0, 10]),
 torch.tensor([10, 10, 10, 12, 10,  4,  2, 15,  8, 15, 14, 10,  6, 10,  6, 10,  6, 10,
          4,  5]),
 torch.tensor([10, 13, 10,  4, 13, 10, 10,  5, 10,  6, 10,  4,  5,  5, 10,  5,  5, 10,
          5]),
 torch.tensor([14, 13,  4, 15,  8, 15,  8, 10,  4,  4,  5, 10,  4,  5, 10,  5, 10,  6,
         10, 17,  5,  5, 10,  4,  4, 19, 15,  0, 10]),
 torch.tensor([10,  4,  4, 15, 10,  4,  5, 10,  4,  5, 10,  4,  5, 10,  4,  4,  5,  5,
          5]),
 torch.tensor([10,  8, 15, 14,  4,  5, 10,  4, 15,  8, 10,  4,  4, 15, 15, 15,  8]),
 torch.tensor([14, 10, 10, 10,  4, 17,  5, 10, 15, 14, 10,  0, 14,  4, 14, 10,  4,  4,
         13, 15, 15, 10,  6,  8, 15,  8, 15]),
 torch.tensor([10, 10,  4,  1, 14, 10, 10,  4, 10,  4, 10,  4, 19, 19, 14,  4, 10,  4,
         10,  4,  4,  4,  5, 10,  4, 15]),
 torch.tensor([15, 13, 10,  6,  8, 10,  4,  2, 15,  1, 15, 15, 10,  1, 14,  4, 15, 14,
          5, 10, 10, 10, 10,  1,  7,  0, 15]),
 torch.tensor([10,  4,  0, 10, 10,  0, 10,  4, 13, 10, 10]),
 torch.tensor([10, 10, 13, 15, 14, 10, 15, 14,  4, 10,  6, 10, 10,  4, 13,  4, 13, 10,
          4,  5]),
 torch.tensor([10,  4, 14,  5,  5,  5,  5,  5,  5]),
 torch.tensor([10, 10,  7, 10,  8, 15, 14, 10, 15, 14, 10,  5,  5,  5, 10]),
 torch.tensor([17,  5, 10, 10,  6, 10,  5,  5, 10,  4,  5,  4,  4,  5,  5,  5]),
 torch.tensor([10,  4, 15, 14, 10,  5,  6, 15,  8, 14, 10,  5, 10,  5,  6, 10,  5]),
 torch.tensor([10,  4,  4, 15,  8, 15,  7, 10,  4, 15, 14,  4, 13, 10,  4,  5,  4, 10,
          5,  6,  5,  5, 10,  5,  6, 15,  8, 15,  8, 10,  0, 10,  4, 19, 15,  6,
         10,  6, 10,  5]),
 torch.tensor([10,  4, 19, 15, 15,  7, 14,  4, 14,  4, 14, 18, 10,  4,  1, 15, 10, 10,
         10, 10, 10, 10,  6,  6, 10, 10,  6, 10,  4,  6, 10,  6, 10,  6, 10,  0,
         10, 10,  6, 10, 10,  8, 10,  9, 14, 10,  9,  5,  4]),
 torch.tensor([14, 10,  0, 10,  4,  0, 10,  4,  4,  5,  4,  4,  4,  4,  8, 10, 10,  0,
         10,  4,  6, 10, 10,  6, 10,  4, 10,  4,  9,  5, 10,  4, 10,  0, 10,  0,
         10,  6, 10, 10, 10,  0,  0, 10,  0, 10,  0, 10,  7, 10,  4]),
 torch.tensor([10,  6,  8, 15,  8, 15,  0, 15, 18,  1,  8, 15, 14]),
 torch.tensor([10,  4, 14, 11,  8, 18, 14, 14, 14,  5]),
 torch.tensor([10, 12, 10,  4,  8, 15, 14,  5,  5]),
 torch.tensor([10,  4, 10,  0, 10,  0, 10, 10,  0, 15, 10, 10, 10,  4, 10,  4,  0, 10,
          0, 10,  0, 10,  0, 10,  4,  9, 10,  6, 10,  5]),
 torch.tensor([10, 10,  5, 10, 10,  0, 10,  5, 10,  5, 10,  5, 10,  5, 10,  4,  8, 15,
          8, 17]),
 torch.tensor([ 8,  8, 10, 10, 10,  6, 10,  2,  8, 15, 10,  6, 10,  0,  8, 15, 14, 10,
         10, 10,  2, 10,  6, 10, 18, 10])]

for sentence in range(W_train.shape[0]):
    score_viterbi, backpointers_viterbi = crf.backward_viterbi_log(
        W_train[sentence, :],
        emissions[
            sentence,
        ],
    )
    sequence_viterbi = crf.get_viterbi(backpointers_viterbi)
    sequences_overall += [sequence_viterbi]

assert torch.all(torch.tensor([torch.all(first_batch_sequences_test[ix] == sequences_overall[ix]) for ix in range(len(sequences_overall))]))

## Q3d)

In [ ]:
emissions = crf.calculate_emissions(W_train)

for sentence in range(W_train.shape[0]):
    for word_index in [2, 3, 4]:
        score_viterbi, backpointers_viterbi = crf.backward_viterbi_log(
            W_train[sentence, 1:word_index], emissions[sentence, :word_index]
        )
        #sequence_viterbi = crf.get_viterbi(backpointers_viterbi)
        score_dijkstra, sequence_dijsktra, log_Z = crf.dijkstra_viterbi_log(
            W_train[sentence, 1:word_index], emissions[sentence, :word_index]
        )
        assert torch.isclose(
            score_viterbi,
                score_dijkstra
                + torch.sum(
                    (W_train[sentence, 1:word_index] != crf.eos_idx)
                    * (W_train[sentence, 1:word_index] != crf.pad_idx_word)
                )
                * log_Z,
        )
        #assert torch.all(sequence_viterbi == sequence_dijsktra)

## Q3e)

In [ ]:
%%timeit -n 10
score_viterbi, backpointers_viterbi = crf.backward_viterbi_log(W_train[0, 1:4], emissions[0, :])
#sequence_viterbi = crf.get_viterbi(backpointers_viterbi)

28 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit -n 10
score_dijkstra, sequence_dijsktra, log_Z = crf.dijkstra_viterbi_log(W_train[0, 1:4], emissions[0, :])

53.8 ms ± 742 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit -n 10
score_naive, sequence_naive = crf.viterbi_naive(W_train[0, 1:4], emissions[0, :])

625 ms ± 25.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Q3f)

In [11]:
def train_model_report_accuracy(
    crf,
    lr,
    epochs,
    train_dataloader,
    dev_dataloader,
    pad_token_idx_word,
    pad_token_idx_tag,
):

    """Train model for `epochs` epochs and report performance on 
        dev set after each epoch.

    Parameters
    ---
    crf : NeuralCRF
    lr : float
        Learning rate to train with.
    epochs : int
        For how many epochs to train.
    train_dataloader : torch.DataLoader
    dev_dataloder : torch.DataLoader
    pad_token_idx_word : int
        Index with which to pad the word indices.
    pad_token_idx_tag : int
        Index with which to pad the tag indices.
    """
    crf.train(True)
    optimizer = torch.optim.Adam(crf.parameters(), lr=lr)
    for epoch in range(epochs):
        for i, data in enumerate(train_dataloader):
            torch.autograd.set_detect_anomaly(True)
            W = F.to_tensor(data["words"], padding_value=pad_token_idx_word)
            T = F.to_tensor(data["pos"], padding_value=pad_token_idx_tag)
            for param in crf.parameters():
                param.grad = None
            loss = crf.loss(T, W)
            loss.backward()
            optimizer.step()
        with torch.no_grad():
            predicted_sequences = []
            true_sequences = []
            for i_dev, data_dev in enumerate(valid_dataloader):
                W_dev = F.to_tensor(
                    data_dev["words"], padding_value=pad_token_idx_word
                )
                T_dev = F.to_tensor(
                    data_dev["pos"], padding_value=pad_token_idx_tag
                )
                sequence_viterbi = crf(W_dev)
                predicted_sequences += sequence_viterbi
                for ix in range(W_dev.shape[0]):
                    true_sequences += [
                        T_dev[ix, 1 : (sequence_viterbi[ix].shape[0] + 1)]
                    ]

            acc = torch.tensor(0.0)
            for ix in range(len(predicted_sequences)):
                acc += torch.mean(
                    (predicted_sequences[ix] == true_sequences[ix]).float()
                )
            acc = acc / len(predicted_sequences)
            print("-------------------------")
            print(f"Epoch: {epoch + 1} / {epochs}")
            print(f"Development set accuracy: {acc}")
            print("-------------------------")
        epoch += 1
    return None


In [ ]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

crf = NeuralCRF(
    pad_idx_word=pad_token_idx,
    pad_idx_pos=pos_vocab[pad_token],
    bos_idx=init_token_idx,
    eos_idx=sep_token_idx,
    bot_idx=pos_vocab[init_token],
    eot_idx=pos_vocab[sep_token],
    t_cal=T_CAL,
    transformer=bert,
)
train_model_report_accuracy(
    crf,
    LR,
    EPOCHS,
    train_dataloader,
    valid_dataloader,
    pad_token_idx,
    pos_vocab[pad_token],
)


-------------------------
Epoch: 1 / 3
Development set accuracy: 0.8845860958099365
-------------------------
-------------------------
Epoch: 2 / 3
Development set accuracy: 0.9055859446525574
-------------------------
-------------------------
Epoch: 3 / 3
Development set accuracy: 0.9071151614189148
-------------------------


## Q3g)

In [ ]:
emissions = crf.calculate_emissions(W_train)

for sentence in range(W_train.shape[0]):
    for word_index in [2, 3, 4]:
        assert torch.isclose(
            crf.entropy_naive(
                W_train[sentence, :word_index],
                emissions[
                    sentence,
                ],
            ),
            crf.backward_entropy(W_train[:, 1:word_index], emissions)[sentence],
        )


In [12]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

entropy_regularized_crf = NeuralCRF(
    pad_idx_word=pad_token_idx,
    pad_idx_pos=pos_vocab[pad_token],
    bos_idx=init_token_idx,
    eos_idx=sep_token_idx,
    bot_idx=pos_vocab[init_token],
    eot_idx=pos_vocab[sep_token],
    t_cal=T_CAL,
    transformer=bert,
    beta=10.0,
)
train_model_report_accuracy(
    entropy_regularized_crf,
    LR,
    EPOCHS,
    train_dataloader,
    valid_dataloader,
    pad_token_idx,
    pos_vocab[pad_token],
)


-------------------------
Epoch: 1 / 3
Development set accuracy: 0.07733473926782608
-------------------------
-------------------------
Epoch: 2 / 3
Development set accuracy: 0.07733473926782608
-------------------------
-------------------------
Epoch: 3 / 3
Development set accuracy: 0.07733473926782608
-------------------------


In [13]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

entropy_regularized_crf = NeuralCRF(
    pad_idx_word=pad_token_idx,
    pad_idx_pos=pos_vocab[pad_token],
    bos_idx=init_token_idx,
    eos_idx=sep_token_idx,
    bot_idx=pos_vocab[init_token],
    eot_idx=pos_vocab[sep_token],
    t_cal=T_CAL,
    transformer=bert,
    beta=1.0,
)
train_model_report_accuracy(
    entropy_regularized_crf,
    LR,
    EPOCHS,
    train_dataloader,
    valid_dataloader,
    pad_token_idx,
    pos_vocab[pad_token],
)


-------------------------
Epoch: 1 / 3
Development set accuracy: 0.07733473926782608
-------------------------
-------------------------
Epoch: 2 / 3
Development set accuracy: 0.07733473926782608
-------------------------
-------------------------
Epoch: 3 / 3
Development set accuracy: 0.07733473926782608
-------------------------


In [14]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

entropy_regularized_crf = NeuralCRF(
    pad_idx_word=pad_token_idx,
    pad_idx_pos=pos_vocab[pad_token],
    bos_idx=init_token_idx,
    eos_idx=sep_token_idx,
    bot_idx=pos_vocab[init_token],
    eot_idx=pos_vocab[sep_token],
    t_cal=T_CAL,
    transformer=bert,
    beta=0.1,
)
train_model_report_accuracy(
    entropy_regularized_crf,
    LR,
    EPOCHS,
    train_dataloader,
    valid_dataloader,
    pad_token_idx,
    pos_vocab[pad_token],
)


-------------------------
Epoch: 1 / 3
Development set accuracy: 0.07733473926782608
-------------------------
-------------------------
Epoch: 2 / 3
Development set accuracy: 0.07733473926782608
-------------------------
-------------------------
Epoch: 3 / 3
Development set accuracy: 0.07733473926782608
-------------------------
